<a href="https://colab.research.google.com/github/w95wayne10/personal_financial_management/blob/master/DH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0:  
0:  
[index:(0, 0), 
h**2*1e5=10.96, 
mu=-5.48, 
a=0, 
eta=3, 
pu=0.3682, 
pm=0.2591, 
pd=0.3727, 
price=0.56263, 
delta=0.50304]  

1:  
-3:  
[index:(1, -3), 
h**2*1e5=11.1079, 
mu=-5.554, 
a=0, 
eta=3, 
pu=0.3732, 
pm=0.2491, 
pd=0.3777, 
price=0.0, 
delta=0.0]  
0:  
[index:(1, 0), 
h**2*1e5=10.5215, 
mu=-5.2608, 
a=0, 
eta=3, 
pu=0.3535, 
pm=0.2888, 
pd=0.3578, 
price=0.43253, 
delta=0.50304]  
3:  
[index:(1, 3), 
h**2*1e5=11.1186, 
mu=-5.5593, 
a=0, 
eta=3, 
pu=0.3735, 
pm=0.2484, 
pd=0.3781, 
price=1.22372, 
delta=1.0]  

2:  

-6:  
[index:(2, -6), 
h**2*1e5=11.241, 
mu=-5.6205, 
a=0, 
eta=3, 
pu=0.3776, 
pm=0.2401, 
pd=0.3822, 
price=0, 
delta=inf]  

-3:  
[index:(2, -3), 
h**2*1e5=10.6546, 
mu=-5.3273, 
a=0, 
eta=3, 
pu=0.3579, 
pm=0.2798, 
pd=0.3623, 
price=0, 
delta=inf,  
index:(2, -3), 
h**2*1e5=10.7135, 
mu=-5.3567, 
a=0, 
eta=3, 
pu=0.3599, 
pm=0.2758, 
pd=0.3643, 
price=0, 
delta=inf]  

0:  
[index:(2, 0), 
h**2*1e5=10.1269, 
mu=-5.0634, 
a=0, 
eta=3, 
pu=0.3402, 
pm=0.3154, 
pd=0.3444, 
price=0.0, 
delta=inf,  
index:(2, 0), 
h**2*1e5=11.2518, 
mu=-5.6259, 
a=0, 
eta=3, 
pu=0.378, 
pm=0.2394, 
pd=0.3826, 
price=0.0, 
delta=inf]  

3:  
[index:(2, 3), 
h**2*1e5=10.6642, 
mu=-5.3321, 
a=0, 
eta=3, 
pu=0.3582, 
pm=0.2791, 
pd=0.3626, 
price=1.22372, 
delta=inf,  
index:(2, 3), 
h**2*1e5=10.7237, 
mu=-5.3619, 
a=0, 
eta=3, 
pu=0.3602, 
pm=0.2751, 
pd=0.3647, 
price=1.22372, 
delta=inf]  

6:  
[index:(2, 6), 
h**2*1e5=11.2614, 
mu=-5.6307, 
a=0, 
eta=3, 
pu=0.3783, 
pm=0.2388, 
pd=0.3829, 
price=2.46242, 
delta=inf]  

A hedger is short 10,000 European calls.  
As an option covers 100 shares of stock, N = 1,000,000.  


0:
0:
[index:(0, 0), 
h**2*1e5=10.96, 
mu=-5.48, 
a=0, 
eta=3, 
pu=0.3682, 
pm=0.2591, 
pd=0.3727, 
price=0.56263, 
delta=0.50304]

In [1]:
N = 1000000
delta = 0.50304
stock_number = N*delta